In [30]:
import polars as pl
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import math

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
PICARRO_DATA_DIRECTORy = os.environ.get("PICARRO_DATA_DIRECTORY")

In [31]:
# 10m preprocessed acropolis data
df_acropolis = pl.read_parquet(os.path.join(DATA_DIRECTORY, "processed", "1h_warm_sbs_acropolis.parquet"))

In [32]:
system_ids = df_acropolis["system_id"].unique().to_list()

In [33]:
def plot_acropolis_scatter(x_col_name, y_col_name="diff",max_col = 3, x_title="x_name", y_title="ACROPOLIS - Picarro (ppm)", title_text="title_name"):
    # fixed
    titles = []
    max_rows = math.ceil(len(system_ids)/max_col)

    for system_id in system_ids:
        titles.append(f"System: {int(system_id)}")
        
    fig = make_subplots(rows=max_rows, cols=max_col, shared_yaxes='all', subplot_titles=(titles), x_title=x_title, y_title=y_title)

    row = 1
    col = 0

    for system_id in system_ids:
        
        col += 1
        
        if col > max_col:
            col = 1
            row += 1

        #----
        df_plot = df_acropolis.filter(pl.col("system_id")==system_id) \
            .select(["creation_timestamp", y_col_name, x_col_name]) \
            
        l_delta = df_plot.select(y_col_name).to_series().to_list()
        l_humidity = df_plot.select(x_col_name).to_series().to_list()
        
        fig.add_trace(go.Scatter(x=l_humidity, y=l_delta, name = f"MC-{int(system_id)}", mode='markers'),row=row, col=col)

    fig.update_layout(title_text=title_text, height=1000, width=1000, showlegend=False) 

    fig.show()

In [34]:
df_acropolis.head(3)

system_id,sys_name_short,creation_timestamp,system_name,gmp343_raw,gmp343_compensated,gmp343_filtered,gmp343_temperature,sht45_humidity,sht45_temperature,bme280_humidity,bme280_temperature,bme280_pressure,revision,receipt_timestamp,h2o_ah,h2o_v%,gmp343_dry,slope,intercept,wxt532_speed_avg,wxt532_speed_min,wxt532_speed_max,wxt532_direction_avg,wxt532_direction_min,wxt532_direction_max,wxt532_last_update_time,wxt532_temperature,wxt532_heating_voltage,wxt532_supply_voltage,wxt532_reference_voltage,gmp343_corrected,date,std,gmp343_temperature_change,picarro_corrected,diff
i64,str,"datetime[μs, UTC]",str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,"datetime[ns, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,date,f64,f64,f64,f64
2,"""acropolis-2""",2024-06-05 00:00:00 UTC,null,395.191806,448.476806,448.526111,29.967222,40.72575,26.276111,47.852181,26.288208,941.435736,38.0,2024-06-05 00:30:00.250061824 UTC,12.340481,1.83373,456.90447,1.019087,-7.813731,null,null,null,null,null,null,null,null,null,null,null,457.811469,2024-06-05,2.383963,0.25,457.823437,-0.011968
2,"""acropolis-2""",2024-06-05 01:00:00 UTC,null,393.502778,446.2425,446.139167,29.809444,40.822667,26.201167,48.092806,26.214139,941.533417,38.0,2024-06-05 01:30:04.584747008 UTC,12.264646,1.821325,454.415625,1.019087,-7.813731,null,null,null,null,null,null,null,null,null,null,null,455.27512,2024-06-05,1.928482,0.25,456.011315,-0.736195
2,"""acropolis-2""",2024-06-05 02:00:00 UTC,null,388.209056,439.806222,439.790278,29.603444,40.845272,26.075883,48.127278,26.091333,941.693428,38.0,2024-06-05 02:29:59.527216384 UTC,12.135064,1.800551,447.854131,1.019087,-7.813731,null,null,null,null,null,null,null,null,null,null,null,448.58839,2024-06-05,0.847298,0.323333,449.967977,-1.379587


In [35]:
# parameters
x_col_name = "gmp343_corrected"
y_col_name = "picarro_corrected"
max_col = 3
x_title = "ACROPOLIS (ppm)"
y_title = "PICARRO (ppm)"
title_text = "Measured CO2 Concentration" 

plot_acropolis_scatter(x_col_name=x_col_name,y_col_name=y_col_name,max_col=max_col, x_title=x_title, y_title = y_title, title_text=title_text)

In [36]:
# parameters
x_col_name = "gmp343_temperature_change"
max_col = 3
x_title = "Temperature Change (°C)"
title_text = "Scatter Plots: Delta CO2 vs Temperature Change" 

plot_acropolis_scatter(x_col_name=x_col_name,max_col=max_col, x_title=x_title, title_text=title_text)

In [37]:
# parameters
x_col_name = "h2o_v%"
max_col = 3
x_title = "Absolute Humidity (%)"
title_text = "Scatter Plots: Delta CO2 vs Absolute Humidity" 

plot_acropolis_scatter(x_col_name=x_col_name,max_col=max_col, x_title=x_title, title_text=title_text)

In [38]:
# parameters
x_col_name = "gmp343_temperature"
max_col = 3
x_title = "Temperature (°C)"
title_text = "Scatter Plots: Delta CO2 vs Temperature" 

plot_acropolis_scatter(x_col_name=x_col_name,max_col=max_col, x_title=x_title, title_text=title_text)

In [39]:
# parameters
x_col_name = "bme280_pressure"
max_col = 3
x_title = "Pressure (mbar)"
title_text = "Scatter Plots: Delta CO2 vs Pressure" 

plot_acropolis_scatter(x_col_name=x_col_name,max_col=max_col, x_title=x_title, title_text=title_text)